In [ ]:
# -*- coding: utf-8 -*-
import scrapy
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
class LyricScrapperSpider(CrawlSpider):
    name = 'lyric_scrapper'
    allowed_domains = ['lirik.kapanlagi.com']
    start_urls = ['https://lirik.kapanlagi.com/lagu/a_id']
    rules = (Rule(LinkExtractor(allow=(), restrict_css=(".pagination2 ul li a",".col-lirik .div-horizontal2-list a", "div .letterLinksAnchor")), callback="parse_page", follow=True),)
    def parse_page(self, response):
        song = response.css('.col-lirik').extract_first()
        title= response.css('.head-lirik h5::text').extract_first()
        if len(title) > 0 and len(song) > 0:
            scraped_info = {
                'song' : song,
                'title' : title,
                }
            yield scraped_info

In [124]:
import pandas as pd
import numpy as np
import re
from sklearn import preprocessing
from keras.layers import Dense
from keras.layers import Embedding
from keras import Sequential
import keras
import pickle
from IPython.display import display, HTML

In [125]:
scrape_lyric = pd.read_csv('scrape.csv')
scrape_lyric

,song,title
0,"<div class=""col-lirik pagination2"">\n\t\t\t\t<...",Lirik Lagu Indonesia: Abjad A
1,"<div class=""col-lirik lyrics-body lyric258543""...",\n\t\t\t\t\t\tLirik Lagu Abadinya Cintaku -
2,"<div class=""col-lirik lyrics-body lyric256885""...",\n\t\t\t\t\t\tLirik Lagu Abadinya Cinta -
3,"<div class=""col-lirik lyrics-body lyric263457""...",\n\t\t\t\t\t\tLirik Lagu Abadilah Cinta (duet ...
4,"<div class=""col-lirik lyrics-body lyric274548""...",\n\t\t\t\t\t\tLirik Lagu Abadilah Cinta -
5,"<div class=""col-lirik lyrics-body lyric258795""...",\n\t\t\t\t\t\tLirik Lagu Abadilah -
6,"<div class=""col-lirik lyrics-body lyric276141""...",\n\t\t\t\t\t\tLirik Lagu Abadikan Kisah Ini -
7,"<div class=""col-lirik lyrics-body lyric261665""...",\n\t\t\t\t\t\tLirik Lagu Abadikan Cinta -
8,"<div class=""col-lirik lyrics-body lyric264176""...",\n\t\t\t\t\t\tLirik Lagu Abadikah Tragedi -
9,"<div class=""col-lirik lyrics-body lyric261671""...",\n\t\t\t\t\t\tLirik Lagu Abadi -


In [128]:
scrape_lyric.shape

(2317, 2)

In [75]:
scrape_lyric = scrape_lyric.dropna()
scrape_lyric = scrape_lyric[scrape_lyric.song.str.contains('span')]
scrape_lyric = scrape_lyric[~(scrape_lyric.song.str.contains('Belum Ada Lirik'))]
scrape_lyric.shape

(2317, 2)

In [126]:
def clean_text_lyric(lyric, max_length):
    lyric_sample = lyric.split('</span>')
    lyric_sample = lyric_sample[:-1]
    
    clean_song_lyric = ""
    for sentence in lyric_sample:
        if 'http' not in sentence:
            sentence = re.sub(r'<div .*>','',sentence)
            sentence = re.sub(r'<span .*>','',sentence)
            sentence = sentence.strip()
            if len(sentence) > 0:
                sentence = re.sub(r'[^a-zA-Z0-9<\/>\s]',' ', sentence)

                sentence = " <div> " + sentence + " </div> "
                sentence = re.sub(r'<div>\s+<br>','<br> <div> ', sentence)
                sentence = re.sub(r'(\d|\r|\t|\n)','',sentence)

                clean_song_lyric += sentence

    clean_song_lyric ='<start>' * (max_length-1) + clean_song_lyric
    return clean_song_lyric

In [127]:
#max_length = 10

scrape_lyric = pd.read_csv('scrape.csv')
scrape_lyric = scrape_lyric[['song','title']]
scrape_lyric = scrape_lyric.dropna()
scrape_lyric = scrape_lyric[scrape_lyric.song.str.contains('span')]
scrape_lyric.song = scrape_lyric.song.apply(lambda x: clean_text_lyric(x,max_length))
scrape_lyric = scrape_lyric[~(scrape_lyric.song.str.contains('Belum Ada Lirik'))]
scrape_lyric = scrape_lyric.reset_index(drop=True)
scrape_lyric

,song,title
0,<start><start><start><start><start><start><sta...,\n\t\t\t\t\t\tLirik Lagu Abadinya Cintaku -
1,<start><start><start><start><start><start><sta...,\n\t\t\t\t\t\tLirik Lagu Abadinya Cinta -
2,<start><start><start><start><start><start><sta...,\n\t\t\t\t\t\tLirik Lagu Abadilah Cinta (duet ...
3,<start><start><start><start><start><start><sta...,\n\t\t\t\t\t\tLirik Lagu Abadilah -
4,<start><start><start><start><start><start><sta...,\n\t\t\t\t\t\tLirik Lagu Abadikan Cinta -
5,<start><start><start><start><start><start><sta...,\n\t\t\t\t\t\tLirik Lagu Abadikah Tragedi -
6,<start><start><start><start><start><start><sta...,\n\t\t\t\t\t\tLirik Lagu Abadi -
7,<start><start><start><start><start><start><sta...,\n\t\t\t\t\t\tLirik Lagu Ababli -
8,<start><start><start><start><start><start><sta...,\n\t\t\t\t\t\tLirik Lagu Aankhen Khuli -
9,<start><start><start><start><start><start><sta...,\n\t\t\t\t\t\tLirik Lagu AADD (Antara Aku Dan ...


In [129]:
def generate_train_dataset(song_series, window):
    train_data = []
    train_label = []
    for song_string in song_series:
        song_arr = song_string.split()
        len_song_arr = len(song_arr)

        for i in range(0,len_song_arr-window+1):
            train_data.append(' '.join(song_arr[i:i+window-1]))
            train_label.append(song_arr[i+window-1])
    return np.array(train_data), np.array(train_label)

In [130]:
X, y = generate_train_dataset(scrape_lyric.song, window=max_length)
pre = keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-.:;=?@[\\]^_`{|}~\t\n')
pre.fit_on_texts(X.flatten())

vocab_size = len(pre.word_index)
X = pre.texts_to_sequences(X)
y = pre.texts_to_sequences(y)

In [136]:
model = Sequential()
e = Embedding(vocab_size,100 ,input_length=max_length-1, trainable=True)
model.add(e)
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Bidirectional(keras.layers.CuDNNLSTM(100)))
model.add(keras.layers.BatchNormalization())

model.add(Dense(100, activation='relu'))
model.add(keras.layers.BatchNormalization())

model.add(Dense(vocab_size+1, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [132]:
def batch_data(start,end):
    global X,y,y_input, X_input
    X_input = np.array(X)[start:end]
    y_input = np.array(y)[start:end]
    y_input = keras.utils.to_categorical(y_input,vocab_size+1)

In [133]:
#y = pre.texts_to_array(y)
#y_input = np.array(y)[0:1000]
#y_input = keras.utils.to_categorical(y_input,vocab_size+1)
#y_input.shape
#y = np.array(y,dtype='object')
X

[[154, 1, 37, 26, 20, 15, 20183, 282, 2],
 [1, 37, 26, 20, 15, 20183, 282, 2, 1],
 [37, 26, 20, 15, 20183, 282, 2, 1, 318],
 [26, 20, 15, 20183, 282, 2, 1, 318, 1472],
 [20, 15, 20183, 282, 2, 1, 318, 1472, 2],
 [15, 20183, 282, 2, 1, 318, 1472, 2, 1],
 [20183, 282, 2, 1, 318, 1472, 2, 1, 2010],
 [282, 2, 1, 318, 1472, 2, 1, 2010, 1833],
 [2, 1, 318, 1472, 2, 1, 2010, 1833, 2],
 [1, 318, 1472, 2, 1, 2010, 1833, 2, 1],
 [318, 1472, 2, 1, 2010, 1833, 2, 1, 86],
 [1472, 2, 1, 2010, 1833, 2, 1, 86, 12114],
 [2, 1, 2010, 1833, 2, 1, 86, 12114, 2],
 [1, 2010, 1833, 2, 1, 86, 12114, 2, 1],
 [2010, 1833, 2, 1, 86, 12114, 2, 1, 761],
 [1833, 2, 1, 86, 12114, 2, 1, 761, 4848],
 [2, 1, 86, 12114, 2, 1, 761, 4848, 2],
 [1, 86, 12114, 2, 1, 761, 4848, 2, 1],
 [86, 12114, 2, 1, 761, 4848, 2, 1, 761],
 [12114, 2, 1, 761, 4848, 2, 1, 761, 2688],
 [2, 1, 761, 4848, 2, 1, 761, 2688, 2],
 [1, 761, 4848, 2, 1, 761, 2688, 2, 1],
 [761, 4848, 2, 1, 761, 2688, 2, 1, 2019],
 [4848, 2, 1, 761, 2688, 2, 1, 2019

In [134]:
y

[[1],
 [318],
 [1472],
 [2],
 [1],
 [2010],
 [1833],
 [2],
 [1],
 [86],
 [12114],
 [2],
 [1],
 [761],
 [4848],
 [2],
 [1],
 [761],
 [2688],
 [2],
 [1],
 [2019],
 [189],
 [2],
 [1],
 [792],
 [9],
 [127],
 [2],
 [1],
 [105],
 [111],
 [2],
 [1],
 [8],
 [38],
 [203],
 [2],
 [1],
 [8558],
 [96],
 [2],
 [1],
 [249],
 [156],
 [2],
 [1],
 [468],
 [116],
 [2],
 [1],
 [8],
 [38],
 [680],
 [2],
 [1],
 [12115],
 [480],
 [2],
 [1],
 [86],
 [2253],
 [43],
 [2],
 [1],
 [12116],
 [8559],
 [116],
 [2],
 [1],
 [2026],
 [240],
 [155],
 [2],
 [1],
 [1104],
 [1104],
 [6],
 [570],
 [2],
 [1],
 [305],
 [66],
 [369],
 [480],
 [2],
 [1],
 [4849],
 [282],
 [2],
 [1],
 [318],
 [1472],
 [2],
 [1],
 [2019],
 [189],
 [2],
 [1],
 [792],
 [9],
 [127],
 [2],
 [34],
 [29],
 [84],
 [2],
 [1],
 [19],
 [4],
 [680],
 [523],
 [50],
 [162],
 [4668],
 [9],
 [1554],
 [2],
 [1],
 [29],
 [176],
 [680],
 [33],
 [119],
 [2],
 [1],
 [10],
 [50],
 [1910],
 [884],
 [491],
 [11],
 [13],
 [2254],
 [2],
 [3],
 [1],
 [487],
 [120],
 [750

In [135]:
i = 0
max_epoch = 1 # Change this max_epoch
instance_max = len(X)
batch_size = 10000

for i in range(max_epoch):
    start_idx_counter = 0
    print("epoch %d" % (i))
    while start_idx_counter < instance_max:
        batch_data(start_idx_counter, min(start_idx_counter+batch_size,instance_max))
        model.fit(X_input, y_input, epochs=1, batch_size=64) 
        start_idx_counter += batch_size

epoch 0


C:\Users\user\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
10000/10000 [==============================] - 22s 2ms/step - loss: 6.9976
Epoch 1/1
10000/10000 [==============================] - 20s 2ms/step - loss: 5.3548
Epoch 1/1
10000/10000 [==============================] - 21s 2ms/step - loss: 5.2823
Epoch 1/1
10000/10000 [==============================] - 20s 2ms/step - loss: 5.1877
Epoch 1/1
10000/10000 [==============================] - 21s 2ms/step - loss: 4.5371
Epoch 1/1
10000/10000 [==============================] - 21s 2ms/step - loss: 4.8770
Epoch 1/1
10000/10000 [==============================] - 20s 2ms/step - loss: 5.0342
Epoch 1/1
10000/10000 [==============================] - 20s 2ms/step - loss: 4.9380
Epoch 1/1
10000/10000 [==============================] - 20s 2ms/step - loss: 4.5479
Epoch 1/1
10000/10000 [==============================] - 20s 2ms/step - loss: 4.8668
Epoch 1/1
10000/10000 [==============================] - 20s 2ms/step - loss: 4.2892
Epoch 1/1
10000/10000 [==============================] - 20s 2ms/

InvalidArgumentError:  indices[44,8] = 21242 is not in [0, 21242)
	 [[node embedding_8/embedding_lookup (defined at C:\Users\user\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_keras_scratch_graph_104207]

Function call stack:
keras_scratch_graph


In [123]:
print(X_input[15,8])
print(y_input[15,8])

2
0.0


In [117]:
model.save('lyric_gen_model.h5')
pickle.dump( pre, open( "tokenizer_pre_lyric.p", "wb" ) )

In [111]:
model = keras.models.load_model('lyric_gen_model.h5')
pre = pickle.load( open( "tokenizer_pre_lyric.p", "rb" ) )

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [112]:
def preprocess_predict(X):
    X = pre.texts_to_sequences(X)
    return X

def is_word_in_dict(words, word_idx):
    for word in words.split():
        if word not in word_idx:
            print(word_idx)
            print(word)
            return False
    return True

def generate_song_by_model(model, idx_word, X_sent, total_length, word_idx, total_classes, word_generated=80):
    string_returned = X_sent
    if is_word_in_dict(X_sent, word_idx):
        X = np.array(preprocess_predict([X_sent]))
        counter = 1

        for i in range(word_generated):
            y_pred = model.predict(X)
            rnd_choice = np.random.choice(range(0,total_classes),size=1,replace=False, p=y_pred[0])
            word_predict = idx_word[rnd_choice[0]]
            string_returned = string_returned + " " + word_predict
            X = ' '.join(string_returned.split()[counter:])
            X = np.array(preprocess_predict([X]))
            counter += 1

        return string_returned

In [115]:
max_length = 10
vocab_size = len(pre.word_index)
seed = "<start> " * (max_length-1)
generated_lyric = generate_song_by_model(model, pre.index_word, seed, max_length, pre.word_index, vocab_size+1,word_generated=10)
display(HTML(generated_lyric).data)

{'<div>': 1, '</div>': 2, '<br>': 3, 'yang': 4, 'aku': 5, 'ku': 6, 'kau': 7, 'tak': 8, 'di': 9, 'dan': 10, 'cinta': 11, 'i': 12, 'kita': 13, 'ini': 14, 'endofembeddedmusicvideo': 15, 'you': 16, 'kamu': 17, 'hati': 18, 'ada': 19, '><': 20, 'semua': 21, 'to': 22, 'the': 23, 'lagi': 24, 'jangan': 25, 'embeddedmusicvideo': 26, 'hanya': 27, 'dalam': 28, 'oh': 29, 'bisa': 30, 'ya': 31, 'untuk': 32, 'hatiku': 33, 'dirimu': 34, 'jadi': 35, 'kan': 36, '<': 37, 'pernah': 38, 'ingin': 39, 'reff': 40, 'itu': 41, 'selalu': 42, 'sayang': 43, 'my': 44, 'a': 45, 'dengan': 46, 'apa': 47, 'tapi': 48, 'me': 49, 's': 50, 'and': 51, 'saja': 52, 'hari': 53, 'mau': 54, 'masih': 55, 'akan': 56, 'dari': 57, 'gak': 58, 'rasa': 59, 'sudah': 60, 'it': 61, 'diriku': 62, 'dia': 63, 'engkau': 64, 'malam': 65, 'ke': 66, 'kini': 67, 'saat': 68, 'satu': 69, 'hidup': 70, 'bila': 71, 'tahu': 72, 'harus': 73, 'goyang': 74, 't': 75, 'in': 76, 'pergi': 77, 'tuk': 78, 'sampai': 79, 'telah': 80, 'mungkin': 81, 'bukan': 82, 'w

None

In [114]:
print(generated_lyric)

None
